In [19]:
# !conda install -y torchvision

import cv2
from PIL import Image, ImageOps
import math
import time

# PyTorch Library
import torch
import shutil


# PyTorch Neural Network
import torch.nn as nn

# Allows us to transform data
import torchvision.transforms as transforms

# Allows us to get the digit dataset
from torchvision.datasets import ImageFolder

# Creating graphs
import torchvision.models as models
import matplotlib.pylab as plt
from transformers import pipeline

from tqdm import tqdm


# Allows us to use arrays to manipulate and store data
import numpy as np
import kagglehub
import os

path = kagglehub.dataset_download("karakaggle/kaggle-cat-vs-dog-dataset")
path = os.path.join(path, "kagglecatsanddogs_3367a")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\amann\.cache\kagglehub\datasets\karakaggle\kaggle-cat-vs-dog-dataset\versions\1\kagglecatsanddogs_3367a


In [37]:
mean = [0.485, 0.456, 0.406]

std = [0.229, 0.224, 0.225]

composed = transforms.Compose(
    [
        transforms.Resize(256),  # Resize shortest side to 256
        transforms.CenterCrop(224),  # Crop to 224x224
        transforms.ToTensor(),  # Convert to tensor (scales to [0,1])
        transforms.Normalize(  # Standardize using ImageNet stats
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        ),
    ]
)

In [38]:
is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

print(device)

GPU is available
cuda


In [39]:
train_data = ImageFolder(os.path.join(path, "train"), transform=composed)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)


val_data = ImageFolder(os.path.join(path, "val"), transform=composed)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=32, shuffle=True)

print(train_loader)
print(val_loader)


print(train_data[0][0].shape)
# plt.imshow(train_data[0][0])
# plt.show()

torch.Size([3, 224, 224])


In [40]:
model = models.resnet18(pretrained=True)

d:\projects\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\projects\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# model = linear_model(input_size, output_size)
model = models.resnet18(pretrained=True)
print(model)

model.to("cuda")
lr_scheduler = True

base_lr = 0.001

max_lr = 0.01

momentum = 0.9

model.to("cuda")

learning_rate = 5e-05

print(model.parameters())

optimizer_SGD = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

optimizer_Adam = torch.optim.Adam(model.parameters(), lr=learning_rate)

if lr_scheduler:
    scheduler = torch.optim.lr_scheduler.CyclicLR(
        optimizer_SGD,
        base_lr=base_lr,
        max_lr=max_lr,
        step_size_up=5,
        mode="triangular2",
    )

criterion = nn.CrossEntropyLoss()

epochs = 3


def train_model(
    model, train_loader, val_loader, criterion, optimizer, epochs, print_=True
):
    loss_list = []
    accuracy_list = []
    correct = 0
    # global:val_loader
    n_test = len(val_data)
    accuracy_best = 0

    for epoch in tqdm(range(epochs)):

        loss_sublist = []
        # Loop through the data in loader

        for x, y in train_loader:
            x, y = x.to("cuda"), y.to("cuda")

            model.train()

            z = model(x)
            loss = criterion(z, y)
            loss_sublist.append(loss.data.item())
            loss.backward()
            optimizer.step()

            optimizer.zero_grad()
        print("epoch {} done".format(epoch))

        scheduler.step()
        loss_list.append(np.mean(loss_sublist))
        correct = 0

        for x_test, y_test in val_loader:
            x_test, y_test = x_test.to("cuda"), y_test.to("cuda")
            model.eval()
            z = model(x_test)
            _, yhat = torch.max(z.data, 1)
            correct += (yhat == y_test).sum().item()
        accuracy = correct / n_test
        accuracy_list.append(accuracy)
        if accuracy > accuracy_best:
            accuracy_best = accuracy
            # best_model_wts = copy.deepcopy(model.state_dict())

        if print_:
            print("learning rate", optimizer.param_groups[0]["lr"])
            print(
                "The validaion  Cost for each epoch "
                + str(epoch + 1)
                + ": "
                + str(np.mean(loss_sublist))
            )
            print(
                "The validation accuracy for epoch "
                + str(epoch + 1)
                + ": "
                + str(accuracy)
            )
    # model.load_state_dict(best_model_wts)
    return accuracy_list, loss_list, model


accuracy_list, loss_list, model = train_model(
    model, train_loader, val_loader, criterion, optimizer_SGD, epochs
)
print(accuracy_list)
print(loss_list)

# PlotParameters(model, input_size)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

  0%|          | 0/3 [00:00<?, ?it/s]d:\projects\.venv\Lib\site-packages\PIL\TiffImagePlugin.py:870: UserWarning: Truncated File Read
  warnings.warn(str(msg))


epoch 0 done


 33%|███▎      | 1/3 [04:10<08:21, 250.96s/it]

learning rate 0.002800000000000002
The validaion  Cost for each epoch 1: 0.20157720670212234
The validation accuracy for epoch 1: 0.9881810897435898
epoch 1 done


 67%|██████▋   | 2/3 [05:47<02:39, 159.93s/it]

learning rate 0.0046
The validaion  Cost for each epoch 2: 0.03941988147298849
The validation accuracy for epoch 2: 0.9869791666666666
epoch 2 done


100%|██████████| 3/3 [07:16<00:00, 145.66s/it]

learning rate 0.006400000000000001
The validaion  Cost for each epoch 3: 0.03373696200112765
The validation accuracy for epoch 3: 0.984375
[0.9881810897435898, 0.9869791666666666, 0.984375]
[0.20157720670212234, 0.03941988147298849, 0.03373696200112765]


In [43]:
torch.save(model.state_dict(), "model.pt")